In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

In [2]:
from django.db import connection
connection.connect()

In [3]:
from fotoweb.models import *

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

class SelDriver():
  def __init__(self,dbg=False):
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    if dbg:
        chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
    #Change chrome driver path accordingly
    chrome_driver = "chromedriver.exe"
    self.driver = webdriver.Chrome(chrome_driver, options=chrome_options)
    print (self.driver)
    self.driver.implicitly_wait(5)
    self.vars = {}

In [5]:
driver=SelDriver(True)

C:\Users\user\AppData\Local\Temp\ipykernel_2392\301287167.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(chrome_driver, options=chrome_options)


<selenium.webdriver.chrome.webdriver.WebDriver (session="8853c0da84f145b23adced2997125c2b")>


In [6]:

#ee=driver.driver.find_elements(By.CSS_SELECTOR, '.cm-media-thumb img')
ee=driver.driver.find_elements(By.CSS_SELECTOR, '.cm-media-thumb img')
ee

[<selenium.webdriver.remote.webelement.WebElement (session="8853c0da84f145b23adced2997125c2b", element="cb29482c-c628-4450-b5dc-5587830a953c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8853c0da84f145b23adced2997125c2b", element="5ddb62cb-dd53-4ad4-a5b5-07047bec6990")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8853c0da84f145b23adced2997125c2b", element="64b25fb6-6eee-4807-b37c-22ff7286c86f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8853c0da84f145b23adced2997125c2b", element="2c44b173-72ca-43e8-99cb-15de569e2fb0")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8853c0da84f145b23adced2997125c2b", element="37259d92-7751-449b-945a-cd1f6f51cdcc")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8853c0da84f145b23adced2997125c2b", element="a7a0b299-6e7f-4ccc-afaf-49f16c41249d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8853c0da84f145b23adced2997125c2b", element="e54e6a30-771e-4ad9-ba32-c0

In [7]:
srcs=[]
for e in ee:
    src=e.get_attribute('src')
    r=re.search(r'(\d+)\.jpg',src)
    print(src,r.group(1))
    srcs.append(r.group(1))



https://image.shutterstock.com/image-photo/rich-farmland-woods-scotlands-glen-250nw-2204582729.jpg 2204582729
https://image.shutterstock.com/image-photo/stunning-scottish-sunset-water-reservoir-250nw-2204359325.jpg 2204359325
https://image.shutterstock.com/image-photo/glen-quaich-scotland-contains-lake-250nw-2204092727.jpg 2204092727
https://image.shutterstock.com/image-photo/evening-over-scottish-landscape-small-250nw-2204092511.jpg 2204092511
https://image.shutterstock.com/image-photo/gravel-beach-woodland-faroff-hills-250nw-2204092123.jpg 2204092123
https://image.shutterstock.com/image-photo/stunning-sunset-over-glens-hills-250nw-2204092283.jpg 2204092283
https://image.shutterstock.com/image-photo/water-sky-cloud-resources-plant-250nw-2202938267.jpg 2202938267
https://image.shutterstock.com/image-photo/nearby-lush-trees-meadows-scottich-250nw-2202518543.jpg 2202518543
https://image.shutterstock.com/image-photo/aerial-view-scottish-loch-green-250nw-2202518237.jpg 2202518237
https://i

In [8]:
images=[]
for src in srcs:
    driver.driver.get('https://submit.shutterstock.com/catalog_manager/images/'+src)
    name = driver.driver.find_element(By.CSS_SELECTOR,'.o_List_List_itemText div').text
    description = driver.driver.find_element(By.CSS_SELECTOR,'.o_EditorDescription_EditorDescription_descriptionInput').text
    images.append({
        'name':name,
        'description':description,
        'shutter-id':src
    })
    i=None
    try:
        i=Image.objects.get(name=name)
        print(name,'found exact match')
    except:
        name = driver.driver.find_element(By.CSS_SELECTOR,'.o_List_List_itemText div').text.replace('.jpg','')
        try:
            i=Image.objects.filter(name__contains=name).first()
            print(name,'found by contains')
        except:
            print(name,'NOT found')

    if i and i.shutter:
        print('already shutter')
    elif i:
        i.shutter_dt=i.updated_dt
        i.title=driver.driver.find_element(By.CSS_SELECTOR,'.o_EditorDescription_EditorDescription_descriptionInput').text
        i.shutter=True
        i.shutter_url='https://submit.shutterstock.com/catalog_manager/images/'+src
        i.save()
        print('shutter updated')


22-09-01-DSC07007-1.jpg found exact match
shutter updated
22-09-01-DSC07162_HDR-1.jpg found exact match
shutter updated
22-09-01-DSC07011-1.jpg found exact match
shutter updated
22-09-01-DSC07138-1.jpg found exact match
shutter updated
22-09-02-DSC07222-1.jpg found exact match
shutter updated
22-09-01-DSC07191_HDR-1.jpg found exact match
shutter updated
22-08-19-DJI_0888-1.jpg found exact match
shutter updated
22-08-19-DSC06705-1.jpg found exact match
shutter updated
22-08-19-DJI_0890-1.jpg found exact match
shutter updated
22-08-19-DJI_0889-1.jpg found exact match
shutter updated
22-08-19-DSC06707-1.jpg found exact match
shutter updated
22-08-19-DSC06716-1.jpg found exact match
shutter updated
22-08-19-DSC06714-1.jpg found exact match
shutter updated
22-08-20-DJI_0940-1.jpg found exact match
shutter updated
22-08-20-DJI_0949-1.jpg found exact match
shutter updated
22-08-20-DJI_0953-1.jpg found exact match
shutter updated
22-08-20-DJI_0962-1.jpg found exact match
shutter updated
22-08-

In [ ]:
images

In [ ]:
ii